In [1]:
# Перечень команд, которые необходимо выполнить перед запуском jupyter notebook для включения решателей:
# export PATH=$PATH:/Library/Frameworks/GAMS.framework/Versions/47/Resources/
# export PATH=$PATH:/Library/Frameworks/ampl_macos64/

# Доступные решатели:
# _neos              [-] внутренняя ошибка связанная с работой со строками
# _mock_cbc          [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# glpk               [+] 
# _glpk_shell        [+] 
# _mock_glpk         [-] внутренняя ошибка
# _mock_cplex        [-] внутренняя ошибка
# gurobi_direct      [+] 
# gurobi             [+] 
# _gurobi_shell      [+] 
# baron              [-] внутренняя ошибка
# xpress             [+] 
# ipopt              [- +] работает, но ответ не формируется с целыми значениями
# gurobi_persistent  [+] требует предварительный вызов метода set_instance
# gams               [+]
# _gams_shell        [+]
# xpress_direct      [+] 
# xpress_persistent  [+] требует предварительный вызов метода set_instance
# mpec_nlp           [- +] работает, но ответ не формируется с целыми значениями
# mpec_minlp         [+] 
# appsi_gurobi       [+] 
# gdpopt             [-] требуется дополнительное указание алгоритма через аргумент метода solve         
# gdpopt.gloa        [-] внутренняя ошибка
# gdpopt.lbb         [-] внутренняя ошибка
# gdpopt.loa         [-] внутренняя ошибка
# gdpopt.ric         [-] внутренняя ошибка
# gdpopt.enumerate   [+] 
# mindtpy            [-] внутренняя ошибка
# mindtpy.oa         [-] внутренняя ошибка
# mindtpy.ecp        [+] 
# mindtpy.goa        [-] внутренняя ошибка
# mindtpy.fp         [-] внутренняя ошибка
# multistart         [- +] работает, но ответ не формируется с целыми значениями
# ipopt_v2           [- +] работает, но ответ не формируется с целыми значениями
# gurobi_v2          [+] 
# gurobi_direct_v2   [-] ошибка разрешения зависимостей
# trustregion        [-] для инициализации требуется дополнительный аргумент degrees_of_freedom_variables
# ampl               [-] внутренняя ошибка работы решателя

In [2]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import time
from random import randint, shuffle

In [3]:
class CalculateUsefulFilesAction():
    def __init__(self, useful_requirements, T, D):
        self.useful_requirements = useful_requirements
        self.T = T
        self.D = D

    def calculate(self):
        useful_requirements = self.useful_requirements
        T = self.T
        calculate_dependencies = self.__calculate_dependencies

        useful_files = np.dot(useful_requirements, T)
        m = len(useful_files)
        for i in range(m):
            dependencies = calculate_dependencies(i)
            useful_files += dependencies
        return useful_files

    def __calculate_dependencies(self, i):
        useful_requirements = self.useful_requirements
        T = self.T
        D = self.D
        calculate_dependencies = self.__calculate_dependencies
        
        if i == 0:
            useful_files = np.dot(useful_requirements, T)
            return np.dot(useful_files, D)
        else:
            dependencies = calculate_dependencies(i - 1)
            return np.dot(dependencies, D)

In [4]:
class Checker():
    def __init__(self, constraints, varlist):
        self.constraints = constraints
        self.varlist = varlist

    def check(self, vector, check_function_1, check_function_2):
        constraints = self.constraints
        varlist = self.varlist
        
        result = []
        for x in vector:
            f = varlist.add()
            result.append(f)
            constraint_1 = check_function_1(x, f)
            constraint_2 = check_function_2(x, f)
            constraints.add(constraint_1)
            constraints.add(constraint_2)
        return np.array(result)

In [5]:
class IncludeChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M
        
        check_include_1 = lambda x, f: (f + 1 / M <= x + 1)
        check_include_2 = lambda x, f: (x <= M * f)

        result = checker.check(vector, check_include_1, check_include_2)
        return result

In [6]:
class ImplementChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M

        check_implemented_1 = lambda x, f: (x >= f)
        check_implemented_2 = lambda x, f: (x + 1 / M <= 1 + M * f)

        result = checker.check(vector, check_implemented_1, check_implemented_2)
        return result

In [7]:
class CalculatePluginsAction():
    def __init__(self, calculate_useful_files_action, include_checker, A):
        self.calculate_useful_files_action = calculate_useful_files_action
        self.include_checker = include_checker
        self.A = A

    def calculate(self):
        calculate_useful_files_action = self.calculate_useful_files_action
        include_checker = self.include_checker
        A = self.A

        useful_files = calculate_useful_files_action.calculate()
        plugins = np.dot(useful_files, A)
        result = include_checker.check(plugins)
        return result

In [8]:
class AddMultiplyConstraintsAction():
    def __init__(self, constraints, varlist):
        self.constraints = constraints
        self.varlist = varlist

    def add_multiply_constraints(self, x, y):
        constraints = self.constraints
        varlist = self.varlist
        
        f = varlist.add()
        constraints.add((x + y <= f + 1))
        constraints.add((f <= x))
        constraints.add((f <= y))
        constraints.add((f >= 0))
        return f

In [9]:
class CalculateDeliveryFilesAction():
    def __init__(self, calculate_plugins_action, add_multiply_constraints_action, A):
        self.calculate_plugins_action = calculate_plugins_action
        self.add_multiply_constraints_action = add_multiply_constraints_action
        self.A = A

    def calculate(self):
        calculate_plugins_action = self.calculate_plugins_action
        A = self.A
        bin_multiply = self.__bin_multiply
        
        plugins = calculate_plugins_action.calculate()
        result = bin_multiply(A, plugins)
        return result

    def __bin_multiply(self, matrix, vector):
        add_multiply_constraints_action = self.add_multiply_constraints_action
        
        (rows_count, cols_count) = np.shape(matrix)
        result = []
        for row_number in range(rows_count):
            matrix_row = matrix[row_number]
            terms = []
            for col_number in range(cols_count):
                vector_element = vector[col_number]
                matrix_element = matrix_row[col_number]
                term = add_multiply_constraints_action.add_multiply_constraints(vector_element, matrix_element)
                terms.append(term)
            sum_terms = sum(terms)
            result.append(sum_terms)
        return np.array(result)

In [10]:
class CalculateDeliveryRequirementsAction():
    def __init__(self, calculate_delivery_files_action, implement_checker, T):
        self.calculate_delivery_files_action = calculate_delivery_files_action
        self.implement_checker = implement_checker
        self.T = T

    def calculate(self):
        calculate_delivery_files_action = self.calculate_delivery_files_action
        implement_checker = self.implement_checker
        T = self.T

        delivery_files = calculate_delivery_files_action.calculate()
        delivery_requirements = np.dot(T, delivery_files)
        result = implement_checker.check(delivery_requirements)
        return result

In [11]:
class CalculateEquipmentCostAction():
    def __init__(self, calculate_delivery_requirements_action, P):
        self.calculate_delivery_requirements_action = calculate_delivery_requirements_action
        self.P = P

    def calculate(self):
        calculate_delivery_requirements_action = self.calculate_delivery_requirements_action
        P = self.P

        delivery_requirements = calculate_delivery_requirements_action.calculate()
        costs = np.dot(P, delivery_requirements)
        result = sum(costs)
        return result

In [12]:
class ModelBuilder():
    # М                                   - условно большое число
    # k                                   - число плагинов
    # T (traceability)        (n x m)     - матрица трассируемости требований к ПО на файлы исходного кода
    # D (dependency)          (m x m)     - матрица зависимостей между файлами исходного кода
    # P (price)               (n x n)     - матрица расчета стоимости сопровождения требований в поставке
    # E (equirements)         (e x n)     - матрица потребных комплектаций
    def __init__(self, M, k, T, D, P, E):
        self.M = M
        self.k = k
        self.T = T
        self.D = D
        self.P = P
        self.E = E
    
    def build(self):
        M = self.M
        k = self.k
        
        T = self.T
        D = self.D
        P = self.P
        E = self.E

        m = np.shape(T)[1]
        
        model = pyo.ConcreteModel(name = 'Optimal decomposition')
        model.constraints = pyo.ConstraintList()
        model.f = pyo.VarList(domain=pyo.Binary)
        set_m = pyo.Set(initialize=range(m))
        set_k = pyo.Set(initialize=range(k))
        # A (allocation)          (m x k)     - матрица распределения файлов исходного кода по плагинам
        model.A = pyo.Var(set_m, set_k, domain=pyo.Binary)
        A = np.array(model.A)

        for row in A:
            model.constraints.add((sum(row) == 1))

        equipment_costs = []
        for useful_requirements in E:
            calculate_useful_files_action = CalculateUsefulFilesAction(useful_requirements, T, D)
            checker = Checker(model.constraints, model.f)
            include_checker = IncludeChecker(checker, M)
            calculate_plugins_action = CalculatePluginsAction(calculate_useful_files_action, include_checker, A)
            add_multiply_constraints_action = AddMultiplyConstraintsAction(model.constraints, model.f)
            calculate_delivery_files_action = CalculateDeliveryFilesAction(calculate_plugins_action, add_multiply_constraints_action, A)
            implement_checker = ImplementChecker(checker, M)
            calculate_delivery_requirements_action = CalculateDeliveryRequirementsAction(calculate_delivery_files_action, implement_checker, T)
            calculate_equipment_cost_action = CalculateEquipmentCostAction(calculate_delivery_requirements_action, P)
            
            equipment_cost = calculate_equipment_cost_action.calculate()
            equipment_costs.append(equipment_cost)
            
        model.OBJ = pyo.Objective(expr = sum(equipment_costs), sense=pyo.minimize)
        return model

In [13]:
class SolveAction():
    def __init__(self, model, solver_name):
        self.model = model
        self.solver_name = solver_name

    def solve(self):
        solver_name = self.solver_name
        model = self.model
        
        start_time = time.time()
        solver = pyo.SolverFactory(solver_name)
        instance = model.create_instance()
        result = solver.solve(instance)
        end_time = time.time()
        duration = end_time - start_time
        
        #instance.A.display()
        print('Calculation duration (sec):', duration)

In [14]:
class GeneratorT():
    # min_value - минимальное число файлов, которые реализуют одно требование
    # max_value - максимальное число файлов, которые реализуют одно требование
    def __init__(self, n, m, min_value, max_value):
        self.n = n
        self.m = m
        self.min_value = min_value
        self.max_value = max_value

    def generate(self):
        n = self.n
        m = self.m
        get_traced_data = self.__get_traced_data

        result = []
        for _ in range(n):
          data = get_traced_data()
          count = data[0]
          value = data[1]
    
          traced_list = np.zeros(m)
          for i in range(m):
            if i < count:
              traced_list[i] = value
          shuffle(traced_list)
          result.append(traced_list)
        return np.array(result)

    def __get_traced_data(self):
        min_value = self.min_value
        max_value = self.max_value

        count = 0
        value = 0
        while True:
          count = randint(min_value, max_value)
          value = 1 / count
          if count * value == 1:
            break
        return (count, value)

In [15]:
class GeneratorD():
    # min_value - минимальное число файлов зависимостей
    # max_value - максимальное число файлов зависимостей
    def __init__(self, m, min_value, max_value):
        self.m = m
        self.min_value = min_value
        self.max_value = max_value

    def generate(self):
        m = self.m
        min_value = self.min_value
        max_value = self.max_value

        result = []
        for _ in range(m):
          count = randint(min_value, max_value)
          dependencies = np.zeros(m)
          for i in range(m):
            if i < count:
              dependencies[i] = 1
          shuffle(dependencies)
          result.append(dependencies)
        return np.array(result)

In [16]:
class GeneratorP():
    # min_value - минимальное значение стоимости сопровождения единицы функционала
    # max_value - максимальное значение стоимости сопровождения единицы функционала
    def __init__(self, n, min_value, max_value):
        self.n = n
        self.min_value = min_value
        self.max_value = max_value

    def generate(self):
        n = self.n
        min_value = self.min_value
        max_value = self.max_value
    
        result = [[randint(min_value, max_value) for i in range(n)] for j in range(n)]
        return np.array(result)

In [17]:
class GeneratorE():
    # min_value - минимальное число полезных требований в комплектации
    # max_value - максимальное число полезных требований в комплектации
    def __init__(self, e, n, min_value, max_value):
        self.e = e
        self.n = n
        self.min_value = min_value
        self.max_value = max_value

    def generate(self):
        e = self.e
        n = self.n
        min_value = self.min_value
        max_value = self.max_value

        equipments = []
        for equipment_number in range(e):
          total_count = randint(min_value, max_value)
          requirements = np.zeros(n)
          current_count = 0
          for i in range(n):
            if i < total_count:
              requirements[i] = 1
          shuffle(requirements)
          equipments.append(requirements)
        return np.array(equipments)

In [18]:
M = 10 ** 6

In [19]:
e = 1
n = 5
m = 4
k = 3

In [20]:
generator_t = GeneratorT(n, m, 1, 2)
generator_d = GeneratorD(m, 0, 1)
generator_p = GeneratorP(n, -5, 10)
generator_e = GeneratorE(e, n, 1, 3)

In [21]:
T = generator_t.generate()

In [22]:
D = generator_d.generate()

In [23]:
P = generator_p.generate()

In [24]:
E = generator_e.generate()

In [25]:
model_builder = ModelBuilder(M, k, T, D, P, E)
model = model_builder.build()

In [26]:
with_glpk = SolveAction(model, 'glpk')
with_glpk.solve()

Calculation duration (sec): 0.04950308799743652


In [27]:
with_gurobi = SolveAction(model, 'gurobi')
with_gurobi.solve()

Calculation duration (sec): 0.15387582778930664


In [28]:
with_xpress = SolveAction(model, 'xpress')
with_xpress.solve()

Calculation duration (sec): 0.06865310668945312


/Users/dambr/venv/lib/python3.12/site-packages/pyomo/solvers/plugins/solvers/xpress_direct.py:650: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Users/dambr/venv/lib/python3.12/site-packages/xpress/license/community-xpauth.xpr')

  self._solver_model = xpress.problem(name=model.name)


In [29]:
with_gams = SolveAction(model, 'gams')
with_gams.solve()

Calculation duration (sec): 0.31570982933044434


In [30]:
with_mpec_minlp = SolveAction(model, 'mpec_minlp')
with_mpec_minlp.solve()

the transformation: None
Calculation duration (sec): 0.05051708221435547


In [31]:
with_gdpopt = SolveAction(model, 'gdpopt.enumerate')
with_gdpopt.solve()

Calculation duration (sec): 0.21007895469665527


In [32]:
with_mindtpy = SolveAction(model, 'mindtpy.ecp')
with_mindtpy.solve()

Calculation duration (sec): 0.8743267059326172
